In [16]:
df = catalog.load('all_texts_parquet')
df[df['id'] == 9739]['competencias'][0]

[11/12/23 23:16:54] INFO     Loading data from 'all_texts_parquet' (ParquetDataset)...          ]8;id=547612;file://C:\Users\ramon.boucas\Miniconda3\lib\site-packages\kedro\io\data_catalog.py\data_catalog.py]8;;\:]8;id=732770;file://C:\Users\ramon.boucas\Miniconda3\lib\site-packages\kedro\io\data_catalog.py#503\503]8;;\


array([{'competencia': 'Adequação ao Tema', 'motivo': '2.0', 'nota': 'Avalia se o texto consegue explorar as possibilidades de ideias que o tema favorece. Como no vestibular, a redação que foge ao tema é zerada.'},
       {'competencia': 'Adequação e Leitura Crítica da Coletânea', 'motivo': '1.5', 'nota': 'Avalia se o texto consegue perceber os pressupostos da coletânea, assim como fazer relação entre os pontos de vista apresentados e outras fontes de referência.'},
       {'competencia': 'Adequação ao Gênero Textual', 'motivo': '2.0', 'nota': 'Avalia se o texto emprega de forma adequada as características do gênero textual e se consegue utilizá-las de forma consciente e enriquecedora a serviço do projeto de texto.'},
       {'competencia': 'Adequação à modalidade padrão da língua', 'motivo': '1.5', 'nota': 'Avalia se o texto possui competência na modalidade escrita. Dessa forma, verifica o domínio morfológico, sintético, semântico e ortográfico.'},
       {'competencia': 'Coesão e Co

In [9]:
df['competencias'][0][0][0]


0    [{'competencia': 'Adequação ao Tema', 'motivo'...
0    [{'competencia': 'Adequação ao Tema', 'motivo'...
0    [{'competencia': 'Adequação ao Tema', 'motivo'...
0    [{'competencia': 'Adequação ao Tema', 'motivo'...
0    [{'competencia': 'Adequação ao Tema', 'motivo'...
                           ...                        
0    [{'competencia': 'Adequação ao Tema', 'motivo'...
0    [{'competencia': 'Adequação ao Tema', 'motivo'...
0    [{'competencia': 'Adequação ao Tema', 'motivo'...
0    [{'competencia': 'Adequação ao Tema', 'motivo'...
0    [{'competencia': 'Adequação ao Tema', 'motivo'...
Name: competencias, Length: 164, dtype: object